In [136]:
import numpy as np
import pandas as pd
import pickle

In [137]:
icu_stays = pd.read_pickle('pickle_vars/icustays.pkl.xz')
patients = pd.read_pickle('pickle_vars/patients.pkl.xz')
admissions = pd.read_pickle('pickle_vars/admissions.pkl.xz')

In [138]:
data = icu_stays[((icu_stays.FIRST_CAREUNIT == 'MICU') | (icu_stays.FIRST_CAREUNIT == 'SICU')) & (icu_stays.DBSOURCE == 'metavision')][["SUBJECT_ID", "HADM_ID","ICUSTAY_ID","INTIME","OUTTIME"]]
data = data.merge(admissions[["HOSPITAL_EXPIRE_FLAG","DEATHTIME","HADM_ID"]], on='HADM_ID')
data = data.merge(patients, on='SUBJECT_ID') 

In [139]:
gdata = data

In [140]:
print("There are: ")
print(str(len(gdata['SUBJECT_ID'].unique())) + " patients,")
print(str(len(gdata['HADM_ID'].unique())) + " hospitals admissions, and")
print(str(len(gdata['ICUSTAY_ID'].unique())) + " intensive care stays.")

There are: 
11007 patients,
13748 hospitals admissions, and
14595 intensive care stays.


In [141]:
gdata_grouped = gdata.groupby('HADM_ID', as_index=False).agg(SUBJECT_ID=('SUBJECT_ID', np.mean),
                                                             FIRST_INTIME=('INTIME', min),
                                                             STAYS_COUNT=('ICUSTAY_ID', pd.Series.count),
                                                             #FIRST_STAY=('INTIME', min),
                                                             MORTALITY=('HOSPITAL_EXPIRE_FLAG', max))

In [142]:
# This is a slightly hacky way of getting the first_stay_id
gdata_grouped = gdata_grouped.merge(gdata[["ICUSTAY_ID", "INTIME"]], left_on="FIRST_INTIME", right_on="INTIME")
gdata_grouped.drop(columns=["INTIME"],inplace=True)
gdata_grouped.rename(columns={"ICUSTAY_ID": 'FIRST_STAY_ID'},inplace=True)

In [143]:
ic_count = dict()
first_stays = dict()
intimes = dict()

for index, row in gdata_grouped.iterrows():
    ic_count[row['HADM_ID']] = row['STAYS_COUNT']
    intimes[row['HADM_ID']] = row['FIRST_INTIME']
    first_stays[row['HADM_ID']] = row['FIRST_STAY_ID']

In [144]:
print(len(gdata_grouped))
print(sum(gdata_grouped['MORTALITY']==1))
mortality_list = list(gdata_grouped[gdata_grouped['MORTALITY']==1]['HADM_ID'])

13748
1681


In [145]:
print("%d mortalities have no DEATHTIME recorded." %sum(gdata[gdata['DEATHTIME']==None ]['HOSPITAL_EXPIRE_FLAG']==1))
print(" ")

_mortalities = gdata[gdata['DEATHTIME'].notnull()]
ic_deaths = _mortalities.apply(lambda row: row['ICUSTAY_ID'] if row['DEATHTIME'] <= row['OUTTIME'] else -1, axis=1)
ic_deaths = [i for i in ic_deaths if i!=-1]

print("There are %d recorded death times." %len(_mortalities))
print("Of these, %d occur on ICU." %len(ic_deaths))

0 mortalities have no DEATHTIME recorded.
 
There are 1922 recorded death times.
Of these, 1279 occur on ICU.


In [146]:
## These are dictionaries:
f = open('pickle_vars/first_stays.pkl', 'wb')
pickle.dump(first_stays, f)
f.close()

f = open('pickle_vars/stay_counts.pkl', 'wb')
pickle.dump(ic_count, f)
f.close()

## These are lists:
f = open('pickle_vars/mortalities.pkl', 'wb')
pickle.dump(mortality_list, f)
f.close()

f = open('pickle_vars/icu_deaths.pkl', 'wb')
pickle.dump(ic_deaths, f)
f.close()

In [147]:
more_than_one_stay = (np.asarray(list(ic_count.values()))>1).sum()
more_than_two_stay = (np.asarray(list(ic_count.values()))>2).sum()
print(more_than_one_stay)
print(more_than_two_stay)

742
80


In [148]:
## Readmission rate:
more_than_one_stay/float(len(gdata['HADM_ID'].unique()))

0.0539714867617108

In [149]:
## Overall mortality
len(mortality_list)/float(len(gdata['HADM_ID'].unique()))

0.12227233052080302

In [150]:
## Readmission mortality:
death=0
for mortality in mortality_list:
    if ic_count[mortality]>1:
        death+=1
        
readmit_mort = death/float((np.asarray(list(ic_count.values()))>1).sum())
print(readmit_mort)

0.2601078167115903


In [151]:
## Single stay mortality:
death=0
for mortality in mortality_list:
    if ic_count[mortality]==1:
        death+=1
        
single_mort = death/float((np.asarray(list(ic_count.values()))==1).sum())
print(single_mort)

0.11440873443026296
